# Exploring Models 3rd

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import re

import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
from catboost import CatBoostRegressor, Pool

from sklearn.svm import SVR
from xgboost import XGBRegressor

import random

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import optuna

USE_OPTUNA = False

In [5]:
!python /kaggle/input/xgb-lgbm-cat-5/xgb_lgbm_cat.py

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010862 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1363
[LightGBM] [Info] Number of data points in the train set: 154033, number of used features: 11
[LightGBM] [Info] Start training from score 43824.155038
Training until validation scores don't improve for 200 rounds
[150]	train's l2: 4.74867e+09	valid's l2: 5.66054e+09
[300]	train's l2: 4.38439e+09	valid's l2: 5.67473e+09
Early stopping, best iteration is:
[183]	train's l2: 4.6549e+09	valid's l2: 5.65474e+09
LGBM Fold RMSE: 75198.03022229091
0:	learn: 77841.1784444	test: 80577.0638483	best: 80577.0638483 (0)	total: 61.8ms	remaining: 1m 1s
300:	learn: 65498.4982910	test:

In [6]:
!python /kaggle/input/lgbm-cat-2/lgbm_cat.py

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010574 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1363
[LightGBM] [Info] Number of data points in the train set: 154033, number of used features: 11
[LightGBM] [Info] Start training from score 43824.155038
Training until validation scores don't improve for 200 rounds
[150]	train's l2: 4.74867e+09	valid's l2: 5.66054e+09
[300]	train's l2: 4.38439e+09	valid's l2: 5.67473e+09
Early stopping, best iteration is:
[183]	train's l2: 4.6549e+09	valid's l2: 5.65474e+09
LGBM Fold RMSE: 75198.03022229091
0:	learn: 77841.1784444	test: 80577.0638483	best: 80577.0638483 (0)	total: 63.3ms	remaining: 1m 3s
300:	learn: 65498.4982910	test:

In [7]:
# original_data_path = '/kaggle/input/used-car-price-prediction-dataset/used_cars.csv'
# data_path = '/kaggle/input/playground-series-s4e9/'

# original = pd.read_csv(original_data_path)
# train = pd.read_csv(data_path + 'train.csv')
# test = pd.read_csv(data_path + 'test.csv')
# submission = pd.read_csv(data_path + 'sample_submission.csv')

## XGBoost

In [54]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
Original = pd.read_csv('/kaggle/input/used-car-price-prediction-dataset/used_cars.csv')


Original[['milage', 'price']] = Original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

train = pd.concat([train, Original], ignore_index=True)

In [55]:
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

In [56]:
cat_features = test.select_dtypes('object').columns
# Reference: https://www.kaggle.com/code/ambrosm/pss4e8-eda-which-makes-sense
for feature in cat_features:          
    categories = sorted(list(set(train[feature].dropna())))
    dtype = pd.CategoricalDtype(categories=categories, ordered=False)
    
    train.loc[~train[feature].isin(categories), feature] = np.nan
    test.loc[~test[feature].isin(categories), feature] = np.nan
    
    train[feature] = train[feature].astype(dtype)
    test[feature] = test[feature].astype(dtype)

In [57]:
X_train = train.drop(columns=["id", "price"])
y_train = train["price"]

X_test = test.drop(columns=["id"])

In [58]:
def cross_validate_model_x(model, X_train, y_train, X_test, params, n_splits=5):
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    val_scores = []
    test_preds = np.zeros((len(X_test), n_splits), dtype=np.float32)

    # Cross-validation loop
    for fold, (train_ind, valid_ind) in enumerate(cv.split(X_train)):
        X_fold_train = X_train.iloc[train_ind]
        y_fold_train = y_train.iloc[train_ind]
        X_val = X_train.iloc[valid_ind]
        y_val = y_train.iloc[valid_ind]

        # Model initialization and training
        clf = model(**params, enable_categorical=True)
        clf.fit(X_fold_train, y_fold_train, eval_set=[(X_val, y_val)], verbose=0, early_stopping_rounds=50)

        # Predict and evaluate
        val_pred = clf.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_pred))
        val_scores.append(rmse)
        test_preds[:, fold] = clf.predict(X_test)

    test_preds = np.mean(test_preds, axis=1)
    return clf, test_preds, np.mean(val_scores)  # Returning the mean RMSE

In [59]:
xgb_params={
    
'lambda': 0.17694956261235095,
 'alpha': 1.3889763984339085,
 'colsample_bytree': 0.7,
 'subsample': 1.0,
 'learning_rate': 0.008,
 'max_depth': 17,
 'random_state': 2020,
 'min_child_weight': 59,
 'n_estimators': 10000,
 'tree_method': 'gpu_hist'
 
}


print('XGBoost Cross-Validation Results:\n')
xgb_model, test_predsx, avg_rmse_xgb = cross_validate_model_x(XGBRegressor, X_train, y_train, X_test, xgb_params)

XGBoost Cross-Validation Results:



In [60]:
avg_rmse_xgb

72609.76512922066

In [61]:
xgb_result =  pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
xgb_result['price'] = test_predsx.astype(np.float32)
xgb_result

,id,price
0,188533,18932.646484
1,188534,83670.929688
2,188535,52087.875000
3,188536,31151.156250
4,188537,30012.619141
...,...,...
125685,314218,33796.226562
125686,314219,54072.843750
125687,314220,21321.498047
125688,314221,16984.417969


## LGBM

In [16]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
Original = pd.read_csv('/kaggle/input/used-car-price-prediction-dataset/used_cars.csv')


Original[['milage', 'price']] = Original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

train = pd.concat([train, Original], ignore_index=True)

In [17]:
def update(df):
    
    t = 100
    
    cat_c = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title']
    re_ = ['model','engine','transmission','ext_col','int_col']
    
    for col in re_:
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
        
    for col in cat_c:
        df[col] = df[col].fillna('missing')
        df[col] = df[col].astype('category')
        
    return df

train  = update(train)
test   = update(test)

In [18]:
X_train = train.drop(columns=["id", "price"])
y_train = train["price"]

X_test = test.drop(columns=["id"])

In [21]:
def cross_validate_model_l(model, X_train, y_train, X_test, params, n_splits=10):
    callbacks = [log_evaluation(period=150), early_stopping(stopping_rounds=200)]
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    val_scores = []
    test_preds = np.zeros((len(X_test), n_splits), dtype=np.float32)

    for fold, (train_ind, valid_ind) in enumerate(cv.split(X_train)):
        X_fold_train = X_train.iloc[train_ind]
        y_fold_train = y_train.iloc[train_ind]
        X_val = X_train.iloc[valid_ind]
        y_val = y_train.iloc[valid_ind]

        clf = model(**params)
        clf.fit(X_fold_train, y_fold_train, eval_set=[(X_val, y_val)], callbacks=callbacks)

        val_pred = clf.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_pred))
        val_scores.append(rmse)
        test_preds[:, fold] = clf.predict(X_test)

    test_preds = np.mean(test_preds, axis=1)
    return clf, test_preds, np.mean(val_scores)

In [22]:
from lightgbm import LGBMRegressor
# params are taken from this notebook: https://www.kaggle.com/code/noodl35/optuna-lgbm-tuning-used-cars
lgb_params = {
                'num_leaves': 426,
                 'max_depth': 20,
                 'learning_rate': 0.011353178352988012,
                 'n_estimators': 10000,
                 'metric': 'rmse',
                 'subsample': 0.5772552201954328,
                 'colsample_bytree': 0.9164865430101521,
                 'reg_alpha': 1.48699088003429e-06,
                 'reg_lambda': 0.41539458543414265,
                 'min_data_in_leaf': 73,
                 'feature_fraction': 0.751673655170548,
                 'bagging_fraction': 0.5120415391590843,
                 'bagging_freq': 2,
                 'random_state': 42,
                 'min_child_weight': 0.017236362383443497,
                 'cat_smooth': 54.81317407769262,
                 'verbose' : -1             # Set to -1 for silent mode, no process information printed
}

print('LightGBM Cross-Validation Results:\n')
lgb_model,test_predsl, avg_rmse_lgb= cross_validate_model_l(LGBMRegressor, X_train, y_train, X_test, lgb_params)

LightGBM Cross-Validation Results:

Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 72236.5
[300]	valid_0's rmse: 71913.9
[450]	valid_0's rmse: 71929
Early stopping, best iteration is:
[362]	valid_0's rmse: 71901.9
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 78470.7
[300]	valid_0's rmse: 78250.4
[450]	valid_0's rmse: 78321.4
Early stopping, best iteration is:
[261]	valid_0's rmse: 78233.8
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 74213.7
[300]	valid_0's rmse: 73896
[450]	valid_0's rmse: 73936.1
Early stopping, best iteration is:
[312]	valid_0's rmse: 73885.6
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 61781.3
[300]	valid_0's rmse: 61414.5
[450]	valid_0's rmse: 61499.7
Early stopping, best iteration is:
[297]	valid_0's rmse: 61413.5
Training until validation scores don't improve for 200 rounds
[150]	valid_0's rmse: 70494.6
[300]	va

In [23]:
avg_rmse_lgb

72342.5950606447

In [24]:
lgb_result =  pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
lgb_result['price'] = test_predsl.astype(np.float32)
lgb_result

,id,price
0,188533,17690.228516
1,188534,78722.164062
2,188535,53061.667969
3,188536,31533.843750
4,188537,28839.781250
...,...,...
125685,314218,30847.910156
125686,314219,53771.667969
125687,314220,20005.833984
125688,314221,15102.117188


## CatBoost

In [25]:
def cross_validate_model_c(model, X_train, y_train, X_test, params, n_splits=10):
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    val_scores = []
    test_preds = np.zeros((len(X_test), n_splits), dtype=np.float32)

    for fold, (train_ind, valid_ind) in enumerate(cv.split(X_train)):
        X_fold_train = X_train.iloc[train_ind]
        y_fold_train = y_train.iloc[train_ind]
        X_val = X_train.iloc[valid_ind]
        y_val = y_train.iloc[valid_ind]

        clf = model(**params)
        clf.fit(X_fold_train, y_fold_train, eval_set=[(X_val, y_val)], verbose=0)

        val_pred = clf.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_pred))
        val_scores.append(rmse)
        test_preds[:, fold] = clf.predict(X_test)

    test_preds = np.mean(test_preds, axis=1)
    return clf, test_preds, np.mean(val_scores)

In [26]:
from catboost import CatBoostClassifier

cat_params = {
    'cat_features':cat_cols,  
    'learning_rate': 0.075,
    'iterations': 5000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'task_type': 'GPU',
    'max_leaves': 512,
    'fold_permutation_block': 64,
    'random_seed': 42,
    'verbose': False                      
}

print('CatBoost Cross-Validation Results:\n')
cat_model, test_predsc, avg_rmse_cat= cross_validate_model_c(CatBoostRegressor, X_train, y_train, X_test, cat_params)

CatBoost Cross-Validation Results:



In [28]:
avg_rmse_cat

72590.30626530075

In [29]:
cat_result =  pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
cat_result['price'] = test_predsc.astype(np.float32)
cat_result

,id,price
0,188533,18276.066406
1,188534,80060.109375
2,188535,56535.820312
3,188536,28516.541016
4,188537,31977.083984
...,...,...
125685,314218,30481.261719
125686,314219,55770.312500
125687,314220,20407.527344
125688,314221,15668.322266


- avg_rmse_xgb: 72609.76512922066
- avg_rmse_lgb: 72342.5950606447
- avg_rmse_cat: 72590.30626530075

In [30]:
Pred = pd.concat([lgb_result,cat_result,xgb_result], axis=1)['price']
print(Pred)
test_preds = test_predsl * 0.7 + test_predsc * 0.1 + test_predsx*0.2

               price         price         price
0       17690.228516  18276.066406  18932.646484
1       78722.164062  80060.109375  83670.929688
2       53061.667969  56535.820312  52087.875000
3       31533.843750  28516.541016  31151.156250
4       28839.781250  31977.083984  30012.619141
...              ...           ...           ...
125685  30847.910156  30481.261719  33796.226562
125686  53771.667969  55770.312500  54072.843750
125687  20005.833984  20407.527344  21321.498047
125688  15102.117188  15668.322266  16984.417969
125689  41535.699219  35069.933594  44022.523438

[125690 rows x 3 columns]


In [31]:
ensemble_sub =  pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
ensemble_sub['price'] = test_preds

ensemble_sub.to_csv('submission_ensemble.csv', index=False)

## Top 5

In [35]:
def read_csv(number: int) -> pd.DataFrame:
    file = pd.read_csv(f'/kaggle/input/top-4-car-prices-used/{number}.csv')  # Use an f-string for cleaner formatting
    return file

csv0 = read_csv(72016)
csv1 = read_csv(72018)
csv2 = read_csv(72020)
csv3 = read_csv(72021)
csv4 = read_csv(72025)

In [36]:
csv = csv0
csv.rename(columns={'price':'72016'}, inplace=True)
csv['72018'] = csv1['price']
csv['72020'] = csv2['price']
csv['72021'] = csv3['price']
csv['72025'] = csv4['price']

csv.head(10).round(0)

,id,72016,72018,72020,72021,72025
0,188533,17286.0,17278.0,17325.0,17253.0,17109.0
1,188534,78406.0,78388.0,78531.0,78342.0,77965.0
2,188535,55893.0,55943.0,55931.0,56057.0,56309.0
3,188536,28963.0,28925.0,28959.0,28845.0,28616.0
4,188537,29857.0,29862.0,29871.0,29912.0,29994.0
5,188538,18667.0,18689.0,18615.0,18770.0,19079.0
6,188539,12248.0,12240.0,12283.0,12253.0,12192.0
7,188540,43399.0,43366.0,43417.0,43398.0,43358.0
8,188541,81726.0,81699.0,81561.0,81698.0,81972.0
9,188542,31785.0,31791.0,31831.0,31732.0,31534.0


In [37]:
csv['price'] = 0.999*csv['72016'] + 0.1*csv['72018'] + 0.0001*csv['72020']+ 0.0001*csv['72021']+ 0.0001*csv['72025']
csv

,id,72016,72018,72020,72021,72025,price
0,188533,17286.205793,17278.279662,17324.902043,17252.808476,17108.621343,19001.916186
1,188534,78406.384175,78388.394866,78530.885844,78342.248924,77964.975084,86190.301088
2,188535,55893.402697,55942.893449,55931.005159,56057.014704,56309.033795,61448.628345
3,188536,28963.187882,28925.059126,28959.438076,28844.935433,28615.930145,31835.372638
4,188537,29856.918937,29862.492397,29870.908127,29911.815200,29993.629346,32822.288893
...,...,...,...,...,...,...,...
125685,314218,28517.042183,28481.697163,28484.390968,28493.702022,28512.324129,31345.243898
125686,314219,51895.372593,51829.165373,51914.048758,51761.255252,51455.668239,57041.906855
125687,314220,20172.305551,20153.927194,20168.307704,20077.553769,19896.045898,22173.540155
125688,314221,15990.390012,15963.589590,15957.349196,15862.217960,15671.955489,17575.507734


In [39]:
top5_sub = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
top5_sub['price'] = csv['price']
top5_sub.to_csv('./submission_top5.csv', index=False)
top5_sub.head()

,id,price
0,188533,19001.916186
1,188534,86190.301088
2,188535,61448.628345
3,188536,31835.372638
4,188537,32822.288893


In [63]:
final_sub = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
final_sub['price'] = (ensemble_sub['price']) * 0.4 + 0.6 * top5_sub['price']
final_sub.to_csv("submission.csv", index=False)
final_sub.head()

,id,price
0,188533,18600.068657
1,188534,83652.465809
2,188535,58154.907476
3,188536,31563.453075
4,188537,31448.604781
